# Deploiement d'une pipeline d'item similarity

Ce notebook a pour but de créer un ensemble triton pour déployer une pipeline d'item similarity


In [3]:
#%pip install "feast<0.31" faiss-gpu
#!pip install seedir

In [1]:
import os
import numpy as np
import pandas as pd
import feast
import seedir as sd
from nvtabular import ColumnSchema, Schema
from nvtabular import ColumnSelector, Workflow, Dataset
from nvtabular.ops import Operator

from merlin.systems.dag.ensemble import Ensemble
from merlin.systems.dag.ops.softmax_sampling import SoftmaxSampling
from merlin.systems.dag.ops.tensorflow import PredictTensorflow
from merlin.systems.dag.ops.unroll_features import UnrollFeatures
from merlin.systems.triton.utils import send_triton_request
from merlin.systems.dag.ops.workflow import TransformWorkflow

/usr/local/lib/python3.10/dist-packages/tensorflow/__init__.py:29: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils as _distutils
2024-09-04 09:00:45.338569: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-04 09:00:45.481575: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load s

In [2]:
feast

<module 'feast' from '/usr/local/lib/python3.10/dist-packages/feast/__init__.py'>

## Enregistrement des embeddings dans le feature store

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/root/Data/Row/")
DATA_FOLDER = os.environ.get("DATA_FOLDER", "/root/Data/")
MODELS_FOLDER = os.environ.get("MODELS", "/root/Models/")
PROCESSED_FOLDER = os.environ.get("PROCESSED_FOLDER", "/root/Data/Processed/")
feature_repo_path = os.environ.get("FEAST_PATH", "/root/Data/feast_repo/feature_repo")

In [4]:
from merlin.core.dispatch import get_lib
df_lib = get_lib()
df_lib

<module 'cudf' from '/usr/local/lib/python3.10/dist-packages/cudf/__init__.py'>

In [5]:
#feast_embeddings = pd.read_csv('/root/Data/item_embeddings_for_similarity/feast_items_embeddings.csv')
faiss_embeddings = pd.read_csv('/root/Data/item_embeddings_for_similarity/faiss_items_embeddings.csv')
faiss_embeddings = faiss_embeddings[['item_id',	'embedding']]
faiss_embeddings.dtypes

item_id       int64
embedding    object
dtype: object

In [6]:
from datetime import datetime

faiss_embeddings["datetime"] = datetime.now()
faiss_embeddings["datetime"] = faiss_embeddings["datetime"].astype("datetime64[ns]")
faiss_embeddings["created"] = datetime.now()
faiss_embeddings["created"] = faiss_embeddings["created"].astype("datetime64[ns]")

In [7]:
faiss_embeddings.columns = ['item_id_e', 'embedding', 'datetime', 'created']
faiss_embeddings["embedding"] = faiss_embeddings["embedding"].apply(lambda x: np.array(eval(x), dtype=np.float32))

# Vérifiez les types de données
print(faiss_embeddings.dtypes)
faiss_embeddings.head(2)

item_id_e             int64
embedding            object
datetime     datetime64[ns]
created      datetime64[ns]
dtype: object


,item_id_e,embedding,datetime,created
0,108775015,"[0.051989615, -0.008208182, 0.07192026, 0.0078...",2024-09-04 09:00:54.313023,2024-09-04 09:00:54.315691
1,108775044,"[0.027522443, 0.009684976, -0.024995811, 0.030...",2024-09-04 09:00:54.313023,2024-09-04 09:00:54.315691


In [8]:
output_path = '/root/Data/feast_repo/feature_repo/data/item_embeddings.parquet' 
faiss_embeddings.to_parquet(output_path, index=False)

In [9]:
faiss_embeddings.head(2)

,item_id_e,embedding,datetime,created
0,108775015,"[0.051989615, -0.008208182, 0.07192026, 0.0078...",2024-09-04 09:00:54.313023,2024-09-04 09:00:54.315691
1,108775044,"[0.027522443, 0.009684976, -0.024995811, 0.030...",2024-09-04 09:00:54.313023,2024-09-04 09:00:54.315691


In [10]:
f = open(os.path.join(feature_repo_path, "item_embeddings_4_similarity.py"), "w")
f.write(
    """
from datetime import timedelta
from feast import Entity, Field, FeatureView, ValueType
from feast.types import Int32, Float32, Array
from feast.infra.offline_stores.file_source import FileSource

item_embeddings = FileSource(
    path="{}",
    timestamp_field="datetime",
    created_timestamp_column="created",
)

item = Entity(name="item_id_e", value_type=ValueType.INT32,)

item_embeddings_view = FeatureView(
    name="item_embeddings",
    entities=[item],
    ttl=timedelta(0),
    schema=[
        Field(name="embedding", dtype=Array(Float32)),
    ],
    online=True,
    source=item_embeddings,
    tags=dict(),
)
""".format(
        os.path.join(feature_repo_path, "data/", "item_embeddings.parquet")
    )
)
f.close()

In [11]:
import seedir as sd

feature_repo_path = os.path.join(feature_repo_path)
sd.seedir(
    feature_repo_path,
    style="lines",
    itemlimit=10,
    depthlimit=3,
    #exclude_folders=".ipynb_checkpoints",
    sort=True,
)

feature_repo/
├─__init__.py
├─__pycache__/
│ ├─__init__.cpython-310.pyc
│ ├─example_repo.cpython-310.pyc
│ └─test_workflow.cpython-310.pyc
├─cufile.log
├─data/
│ ├─item_embeddings.parquet
│ ├─item_features.parquet
│ ├─online_store.db
│ ├─registry.db
│ └─user_features.parquet
├─feature_store.yaml
├─item_embeddings_4_similarity.py
├─item_features.py
├─test_workflow.py
└─user_features.py


In [12]:
%cd $feature_repo_path
!find . -name ".ipynb_checkpoints" -exec rm -r {} +
!feast apply

/root/Data/feast_repo/feature_repo


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Created entity user_id
Created entity item_id
Created feature view user_features
Created feature view item_features

Created sqlite table feast_repo_item_features
Created sqlite table feast_repo_user_features



In [13]:
!feast materialize 1995-01-01T01:01:01 2025-01-01T01:01:01

Materializing 3 feature views from 1995-01-01 01:01:01+00:00 to 2025-01-01 01:01:01+00:00 into the sqlite online store.

item_embeddings:
100%|██████████████████████████████████████████████████████| 105100/105100 [01:46<00:00, 990.27it/s]
user_features:
100%|█████████████████████████████████████████████████████| 442707/442707 [04:40<00:00, 1578.42it/s]
item_features:
100%|███████████████████████████████████████████████████████| 23417/23417 [00:12<00:00, 1946.19it/s]


In [14]:
feature_store = feast.FeatureStore(feature_repo_path)

## Creation de l'index Faiss

In [15]:
from merlin.systems.dag.ops.faiss import QueryFaiss, setup_faiss

faiss_index_path = os.path.join(DATA_FOLDER, 'faiss_index', "index_item_similarity.faiss")
faiss_embeddings = faiss_embeddings[['item_id_e', 'embedding']]
faiss_embeddings.columns = ['item_id', 'embedding']
setup_faiss(faiss_embeddings, faiss_index_path, embedding_column="embedding")

## Creation de la pipeline d'item similarity

Operateur Faiss personnalisé pour que ça marche

In [16]:
import warnings
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [17]:
#
# Copyright (c) 2022, NVIDIA CORPORATION.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
import os
from pathlib import Path
from shutil import copy2

import faiss
import numpy as np
import cupy

from merlin.core.dispatch import HAS_GPU
from merlin.core.protocols import DataFrameLike, Transformable
from merlin.dag import BaseOperator, ColumnSelector
from merlin.schema import ColumnSchema, Schema


class QueryFaissCustomed(BaseOperator):
    """
    This operator creates an interface between a FAISS[1] Approximate Nearest Neighbors (ANN)
    Index and Triton Infrence Server. The operator allows users to perform different supported
    types[2] of Nearest Neighbor search to your ensemble. For input query vector, we do an ANN
    search query to find the ids of top-k nearby nodes in the index.

    References
    ----------
    [1] https://github.com/facebookresearch/faiss)
    [2] https://github.com/facebookresearch/faiss/wiki/Faiss-indexes
    """

    def __init__(self, index_path, topk=10):
        """
        Creates a QueryFaiss Pipelineable Inference Operator.

        Parameters
        ----------
        index_path : str
            A path to an already setup index
        topk : int, optional
            The number of results we should receive from query to Faiss as output, by default 10
        """
        super().__init__()

        self.index_path = str(index_path)
        self.topk = topk
        self._index = None
        self.logger = logging.getLogger(self.__class__.__name__)

    def load_artifacts(self, artifact_path: str) -> None:
        if artifact_path:
            filename = Path(self.index_path).name
            path_artifact = Path(artifact_path)
            if path_artifact.is_file():
                path_artifact = path_artifact.parent
            full_index_path = str(path_artifact / filename)
        else:
            full_index_path = self.index_path
        index = faiss.read_index(full_index_path)

        if HAS_GPU:
            res = faiss.StandardGpuResources()
            index = faiss.index_cpu_to_gpu(res, 0, index)
        self._index = index

    def save_artifacts(self, artifact_path: str) -> None:
        index_filename = os.path.basename(os.path.realpath(self.index_path))
        new_index_path = Path(artifact_path) / index_filename
        copy2(self.index_path, new_index_path)

    def __getstate__(self) -> dict:
        """Return state of instance when pickled.

        Returns
        -------
        dict
            Returns object state excluding index attribute.
        """
        return {k: v for k, v in self.__dict__.items() if k != "_index"}

    def transform(
        self, col_selector: ColumnSelector, transformable: Transformable
    ) -> Transformable:
        """
        Transform input dataframe to output dataframe using function logic.

        Parameters
        ----------
        df : TensorTable
            Input tensor dictionary, data that will be manipulated

        Returns
        -------
        TensorTable
            Transformed tensor dictionary
        """
        
    
        if isinstance(transformable, dict):
            user_vector = list(transformable.values())[0]
        elif hasattr(transformable, 'to_dict'):
            dict_values = list(transformable.to_dict().values())
            if dict_values and isinstance(dict_values[0], dict):
                user_vector = list(dict_values[0].values())[0]
            else:
                user_vector = dict_values[0]
        else:
            raise ValueError(f"Unexpected input type: {type(transformable)}")
    
        # Convertir en numpy array si ce n'est pas déjà le cas
        if not isinstance(user_vector, (np.ndarray, cupy.ndarray)):
            user_vector = np.array(user_vector)
        
        self.logger.error(f"User vector shape before reshape: {user_vector.shape}")
        self.logger.error(f"User vector ndim before reshape: {user_vector.ndim}")
    
        # S'assurer que user_vector est un tableau 2D
        
        user_vector = user_vector.reshape(1, -1)
    
        self.logger.error(f"User vector shape after reshape: {user_vector.shape}")
        self.logger.error(f"User vector ndim after reshape: {user_vector.ndim}")
        self.logger.error(f"Index dimension: {self._index.d}")
    
        # Vérifier que la dimension correspond à celle attendue par l'index
        if user_vector.shape[1] != self._index.d:
            raise ValueError(f"User vector dimension ({user_vector.shape[1]}) does not match index dimension ({self._index.d}), here vector {user_vector}")
    
        
        _, indices = self._index.search(user_vector, self.topk)

        candidate_ids = np.array(indices).astype(np.int32).flatten()

        return type(transformable)({"candidate_ids": candidate_ids})

    def compute_input_schema(
        self,
        root_schema: Schema,
        parents_schema: Schema,
        deps_schema: Schema,
        selector: ColumnSelector,
    ) -> Schema:
        """
        Compute the input schema of this node given the root, parents and dependencies schemas of
        all ancestor nodes.

        Parameters
        ----------
        root_schema : Schema
            The schema representing the input columns to the graph
        parents_schema : Schema
            A schema representing all the output columns of the ancestors of this node.
        deps_schema : Schema
            A schema representing the dependencies of this node.
        selector : ColumnSelector
            A column selector representing a target subset of columns necessary for this node's
            operator

        Returns
        -------
        Schema
            A schema that has the correct representation of all the incoming columns necessary for
            this node's operator to complete its transform.

        Raises
        ------
        ValueError
            Cannot receive more than one input for this node
        """
        input_schema = super().compute_input_schema(
            root_schema, parents_schema, deps_schema, selector
        )
        return input_schema

    def compute_output_schema(
        self, input_schema: Schema, col_selector: ColumnSelector, prev_output_schema: Schema = None
    ) -> Schema:
        """
        Compute the input schema of this node given the root, parents and dependencies schemas of
        all ancestor nodes.

        Parameters
        ----------
        input_schema : Schema
            The schema representing the input columns to the graph
        col_selector : ColumnSelector
            A column selector representing a target subset of columns necessary for this node's
            operator
        prev_output_schema : Schema
            A schema representing the output of the previous node.

        Returns
        -------
        Schema
            A schema object representing all outputs of this node.
        """
        return Schema(
            [
                ColumnSchema("candidate_ids", dtype=np.int32, dims=(None, self.topk)),
            ]
        )

    def validate_schemas(
        self, parents_schema, deps_schema, input_schema, output_schema, strict_dtypes=False
    ):
        if len(input_schema.column_schemas) > 1:
            raise ValueError(
                "More than one input has been detected for this node,"
                / f"inputs received: {input_schema.column_names}"
            )

'''
def setup_faiss(
    item_vector: DataFrameLike,
    output_path: str,
    metric=faiss.METRIC_INNER_PRODUCT,
    item_id_column="item_id",
    embedding_column="embedding",
):
    """
    Utiltiy function that will create a Faiss index from a set of embedding vectors

    Parameters
    ----------
    item_vector : Numpy.ndarray
        This is a matrix representing all the nodes embeddings, represented as a numpy ndarray.
    output_path : string
        target output path
    """
    ids = item_vector[item_id_column].to_numpy().astype(np.int64)
    item_vectors = np.ascontiguousarray(
        np.stack(item_vector[embedding_column].to_numpy()).astype(np.float32)
    )

    index = faiss.index_factory(item_vectors.shape[1], "IVF32,Flat", metric)
    index.nprobe = 8

    index.train(item_vectors)
    index.add_with_ids(item_vectors, ids)
    faiss.write_index(index, str(output_path))
'''

'\ndef setup_faiss(\n    item_vector: DataFrameLike,\n    output_path: str,\n    metric=faiss.METRIC_INNER_PRODUCT,\n    item_id_column="item_id",\n    embedding_column="embedding",\n):\n    """\n    Utiltiy function that will create a Faiss index from a set of embedding vectors\n\n    Parameters\n    ----------\n    item_vector : Numpy.ndarray\n        This is a matrix representing all the nodes embeddings, represented as a numpy ndarray.\n    output_path : string\n        target output path\n    """\n    ids = item_vector[item_id_column].to_numpy().astype(np.int64)\n    item_vectors = np.ascontiguousarray(\n        np.stack(item_vector[embedding_column].to_numpy()).astype(np.float32)\n    )\n\n    index = faiss.index_factory(item_vectors.shape[1], "IVF32,Flat", metric)\n    index.nprobe = 8\n\n    index.train(item_vectors)\n    index.add_with_ids(item_vectors, ids)\n    faiss.write_index(index, str(output_path))\n'

Création de la pipeline

In [18]:
from merlin.core.dispatch import make_df
from merlin.systems.dag.ops.feast import QueryFeast
faiss_index_path = os.path.join(DATA_FOLDER, 'faiss_index', "index_item_similarity.faiss")

#Test input
request = make_df({"item_id_e": [688463003]})
request["item_id_e"] = request["item_id_e"].astype(np.int32)
test_dataset = Dataset(request)

# Embedding retrieval
item_embedding_values = ["item_id_e"] >> QueryFeast.from_feature_view(
    store=feature_store,
    view="item_embeddings",
    column="item_id_e",
    include_id=False,
)

# Similarity comparison
topk_items_similarity = int(
    os.environ.get("topk_items_similarity", "8")
)

similar_items = item_embedding_values >> QueryFaissCustomed(faiss_index_path, topk=topk_items_similarity)



2024-09-04 09:09:16,513 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): usage.feast.dev:443
2024-09-04 09:09:16,520 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): usage.feast.dev:443
2024-09-04 09:09:16,524 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): usage.feast.dev:443


Materializing 1 feature views to 2024-09-04 09:09:16+00:00 into the sqlite online store.

item_embeddings from 2025-01-01 01:01:01+00:00 to 2024-09-04 09:09:16+00:00:


2024-09-04 09:09:16,799 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): usage.feast.dev:443
0it [00:00, ?it/s]
2024-09-04 09:09:19,205 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): usage.feast.dev:443


In [25]:
# Exécuter la pipeline sur les données de test
similarity_workflow = Workflow(similar_items)
output = similarity_workflow.transform(test_dataset)


print(output.to_ddf().compute())

2024-09-04 08:49:37,561 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): usage.feast.dev:443
2024-09-04 08:56:45,619 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): usage.feast.dev:443
2024-09-04 08:56:46,265 - QueryFaissCustomed - ERROR - User vector shape before reshape: (512,)
2024-09-04 08:56:46,266 - QueryFaissCustomed - ERROR - User vector ndim before reshape: 1
2024-09-04 08:56:46,266 - QueryFaissCustomed - ERROR - User vector shape after reshape: (1, 512)
2024-09-04 08:56:46,267 - QueryFaissCustomed - ERROR - User vector ndim after reshape: 2
2024-09-04 08:56:46,267 - QueryFaissCustomed - ERROR - Index dimension: 512


   candidate_ids
0      688463001
1      688463003
2      812371001
3      736870001
4      736870005
5      108775015
6      863937010
7      626366003


Préparation de tensorflow

In [19]:
# prevent TF to claim all GPU memory
from merlin.dataloader.tf_utils import configure_tensorflow
configure_tensorflow()

<function tensorflow.python.dlpack.dlpack.from_dlpack(dlcapsule)>

## Sauvegarde de l'ensemble Triton

In [19]:
if not os.path.isdir("/root/Triton_models"):
    os.makedirs(os.path.join('/root/Triton_models'))

In [20]:
from merlin.core.dispatch import make_df

# create a request to be sent to TIS
request = make_df({"item_id_e": [688463003]})
request["item_id_e"] = request["item_id_e"].astype(np.int32)
print(request)

request_schema = Schema(
    [
        ColumnSchema("item_id_e", dtype=np.int32),
    ]
)
print(request_schema)

   item_id_e
0  688463003
[{'name': 'item_id_e', 'tags': set(), 'properties': {}, 'dtype': DType(name='int32', element_type=<ElementType.Int: 'int'>, element_size=32, element_unit=None, signed=True, shape=Shape(dims=None)), 'is_list': False, 'is_ragged': False}]


Création de l'ensemble

In [21]:
%%time
# define the path where all the models and config files exported to
export_path = os.path.join('/root/Triton_models_test_operateur_perso')

ensemble = Ensemble(similar_items, request_schema)
ens_config, node_configs = ensemble.export(export_path, name='item_similarity')

# return the output column name
outputs = ensemble.graph.output_schema.column_names
print(outputs)

2024-09-04 09:09:19,227 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): usage.feast.dev:443


['candidate_ids']
CPU times: user 7min 29s, sys: 1.81 s, total: 7min 31s
Wall time: 6min 58s


Enlève les fichiers qui pourraient poser problème à Triton

In [23]:
import shutil
export_path = os.path.join('/root/Triton_models_test_operateur_perso')

def remove_checkpoints(dir_path):
    for root, dirs, files in os.walk(dir_path):
        for dir_name in dirs:
            if dir_name == '.ipynb_checkpoints':
                dir_to_remove = os.path.join(root, dir_name)
                print(f"Removing: {dir_to_remove}")
                shutil.rmtree(dir_to_remove)

remove_checkpoints(export_path)

sd.seedir(export_path, style='lines', itemlimit=10, depthlimit=5, sort=True)

Triton_models_test_operateur_perso/
├─0_transformworkflowtriton/
│ ├─1/
│ │ ├─__pycache__/
│ │ │ └─model.cpython-310.pyc
│ │ ├─model.py
│ │ └─workflow/
│ │   ├─categories/
│ │   │ ├─unique.2nd_last_product_code.parquet
│ │   │ ├─unique.2nd_last_product_type.parquet
│ │   │ ├─unique.2nd_popular_department_no.parquet
│ │   │ ├─unique.2nd_popular_product_type.parquet
│ │   │ ├─unique.2nd_popular_section_no.parquet
│ │   │ ├─unique.Active.parquet
│ │   │ ├─unique.FN.parquet
│ │   │ ├─unique.club_member_status.parquet
│ │   │ ├─unique.fashion_news_frequency.parquet
│ │   │ └─unique.last_product_code.parquet
│ │   ├─metadata.json
│ │   └─workflow.pkl
│ └─config.pbtxt
├─1_predicttensorflowtriton/
│ ├─1/
│ │ └─model.savedmodel/
│ │   ├─.merlin/
│ │   │ ├─input_schema.json
│ │   │ └─output_schema.json
│ │   ├─assets/
│ │   ├─fingerprint.pb
│ │   ├─keras_metadata.pb
│ │   ├─saved_model.pb
│ │   └─variables/
│ │     ├─variables.data-00000-of-00001
│ │     └─variables.index
│ └─config.pbtxt
├─2_tr

## Démarrage du Triton Server
Executer dans un terminal : 

tritonserver --model-repository=/root/Triton_models/ --backend-config=tensorflow,version=2

In [24]:
%%time
from merlin.core.dispatch import make_df

# create a request to be sent to TIS
request = make_df({"item_id_e": [736870005]})
request["item_id_e"] = request["item_id_e"].astype(np.int32)

request_schema = Schema(
    [
        ColumnSchema("item_id_e", dtype=np.int32),
    ]
)
outputs = ['candidate_ids']
response = send_triton_request(request_schema, request, outputs, triton_model='item_similarity')
response

CPU times: user 24.3 ms, sys: 7 µs, total: 24.3 ms
Wall time: 69.7 ms


{'candidate_ids': array([736870001, 736870005, 812371001, 688463001, 688463003, 626366003,
        863937010, 108775015], dtype=int32)}

Test de la première pipeline

In [25]:
%%time
# create a request to be sent to TIS

request = make_df({"user_id": [11]})
request["user_id"] = request["user_id"].astype(np.int32)

outputs = ['ordered_ids', 'ordered_scores']

request_schema = Schema(
    [
        ColumnSchema("user_id", dtype=np.int32),
    ]
)

response = send_triton_request(request_schema, request, outputs, triton_model='executor_model')
response

CPU times: user 13.7 ms, sys: 6.9 ms, total: 20.6 ms
Wall time: 1.62 s


{'ordered_ids': array([[783751003, 887542001, 815542002, 817086002, 812432001, 697498001,
         794054001, 880738003, 882066001, 888507002, 690617002, 697058001]],
       dtype=int32),
 'ordered_scores': array([[0.99187917, 0.5780467 , 0.99725145, 0.7040904 , 0.9999831 ,
         0.98252654, 0.99974936, 0.4671511 , 0.66586274, 0.96997446,
         0.99998343, 0.9998634 ]], dtype=float32)}